# DS-SF-25 | Unit Project 3: Basic Machine Learning Modeling

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [75]:
import os
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn import linear_model
import pylab as pl

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.notebook_repr_html', True)

In [76]:
df = pd.read_csv(os.path.join('..', '..', 'dataset', 'ucla-admissions.csv'))
df.dropna(inplace = True)

df.head()

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0


## Part A.  Frequency Table

> ### Question 1.  Create a frequency table for `prestige` and whether or not an applicant was admitted.

In [77]:
pd.crosstab(df.admit, df.prestige)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part B.  Variable Transformations

> ### Question 2.  Create a one-hot encoding for `prestige`.

In [78]:
df_prestige_dummy = pd.get_dummies(df.prestige, prefix = 'prestige')

> ### Question 3.  How many of these binary variables do we need for modeling?

In [79]:
df_prestige_dummy.sum()

prestige_1.0     61.0
prestige_2.0    148.0
prestige_3.0    121.0
prestige_4.0     67.0
dtype: float64

Answer: We only need three of them.

> ### Question 4.  Why are we doing this?

Answer: 1) if we include all we will introduce a multicollinearity problem because the four variables are correlated. 2) We need a baseline scenario to interpret the binary variables coefficients.

> ### Question 5.  Add all these binary variables in the dataset and remove the now redundant `prestige` feature.

In [80]:
df = df.join(df_prestige_dummy)

In [81]:
df.drop('prestige', axis =1, inplace = True)

In [82]:
df

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0.0,0.0,1.0,0.0
1,1,660.0,3.67,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
395,0,620.0,4.00,0.0,1.0,0.0,0.0
396,0,560.0,3.04,0.0,0.0,1.0,0.0
397,0,460.0,2.63,0.0,1.0,0.0,0.0
398,0,700.0,3.65,0.0,1.0,0.0,0.0


## Part C.  Hand calculating odds ratios

Let's develop our intuition about expected outcomes by hand calculating odds ratios.

> ### Question 6.  Create a frequency table for `prestige = 1` and whether or not an applicant was admitted.

In [83]:
pd.crosstab(df.admit, df['prestige_1.0'][(df['prestige_1.0']==1)])

prestige_1.0,1.0
admit,
0,28
1,33


In [84]:
pd.crosstab(df.admit, df['prestige_1.0'][(df['prestige_1.0']==0)])

prestige_1.0,0.0
admit,
0,243
1,93


> ### Question 7.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the most prestigious undergraduate schools.

In [85]:
# Odds
33/28.

1.1785714285714286

> ### Question 8.  Now calculate the odds of admission for undergraduates who did not attend a #1 ranked college.

In [86]:
93/243.

0.38271604938271603

> ### Question 9.  Finally, what's the odds ratio?

In [87]:
33/28*(243/93.)

2.6129032258064515

> ### Question 10.  Write this finding in a sentenance.

Answer: The odds for getting admitted is almost 3 times higher for candidates from a #1 Rank college.

> ### Question 11.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the least prestigious undergraduate schools.  Then calculate their odds ratio of being admitted to UCLA.  Finally, write this finding in a sentenance.

In [88]:
pd.crosstab(df.admit, df['prestige_4.0'])

prestige_4.0,0.0,1.0
admit,,
0,216,55
1,114,12


In [89]:
#Odds for attending the least prestige school
12/55.

0.21818181818181817

In [90]:
#Odds for not attending the least prestige school
114/216.

0.5277777777777778

In [91]:
#Odds ration
12/55.*(216/114.) -1 

-0.5866028708133972

Answer: The odds for getting admitted is almost 60% lower for candidates from a #4 Rank college than others

## Part C. Analysis using `statsmodel`

> ### Question 12.  Fit a logistic regression model prediting admission into UCLA using `gre`, `gpa`, and the prestige of the undergraduate schools.  Use the highest prestige undergraduate schools as your reference point.

In [92]:
df.columns

Index([u'admit', u'gre', u'gpa', u'prestige_1.0', u'prestige_2.0',
       u'prestige_3.0', u'prestige_4.0'],
      dtype='object')

In [93]:
X = df[['gre', 'gpa', 'prestige_1.0', 'prestige_2.0','prestige_3.0']]
y = df.admit

In [99]:
X

,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0
0,380.0,3.61,0.0,0.0,1.0
1,660.0,3.67,0.0,0.0,1.0
2,800.0,4.00,1.0,0.0,0.0
3,640.0,3.19,0.0,0.0,0.0
4,520.0,2.93,0.0,0.0,0.0
...,...,...,...,...,...
395,620.0,4.00,0.0,1.0,0.0
396,560.0,3.04,0.0,0.0,1.0
397,460.0,2.63,0.0,1.0,0.0
398,700.0,3.65,0.0,1.0,0.0


In [104]:
model = sm.Logit(y, X).fit()

Optimization terminated successfully.
         Current function value: 0.605465
         Iterations 5


> ### Question 13.  Print the model's summary results.

In [105]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Mon, 22 Aug 2016   Pseudo R-squ.:                 0.03107
Time:                        13:46:39   Log-Likelihood:                -240.37
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                  0.003913
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0011      0.001      1.043      0.297        -0.001     0.003
gpa             -0.5162      0.192     -2.682      0.007        -0.893    -0.139
prestige_1.0     1.2948      0.385      3.361      0.001         0.540     2.050
prestige_2.0     0.5120      0.327      1.567      0.117        -0.129     1.153
prestige_3.0    -0.0504      0.354     -0.142      0.887        -0.745     0.644
================================================================================
"""

> ### Question 14.  What are the odds ratios of the different features and their 95% confidence intervals?

In [106]:
model.conf_int(.1).\
    rename(columns = {0: '5%', 1: '95%'})

,5%,95%
gre,-0.000616,0.002753
gpa,-0.832801,-0.199567
prestige_1.0,0.661130,1.928474
prestige_2.0,-0.025581,1.049597
prestige_3.0,-0.633144,0.532286


In [107]:
model.params

gre             0.001068
gpa            -0.516184
prestige_1.0    1.294802
prestige_2.0    0.512008
prestige_3.0   -0.050429
dtype: float64

In [108]:
def odds_ratio(x):
    odds_ratio = np.exp(x)
    print odds_ratio 

In [109]:
#gre:
odds_ratio(model.params[1])

0.596793649233


In [110]:
#gpa:
odds_ratio(model.params[2])

3.65027398038


In [111]:
#prestige_1.0:
odds_ratio(model.params[3])

1.66863862319


In [112]:
#prestige_2.0:
odds_ratio(model.params[4])

0.950821160616


In [113]:
#prestige_3.0:
odds_ratio(model.params[5])

IndexError: index out of bounds

> ### Question 15.  Interpret the odds ratio for `prestige = 2`.

In [114]:
np.exp(0.160452)

1.1740414178000445

Answer: Graduated from a prestige =2 college, a candidiate's odds of getting admitted is 1.17 times the odds of candidates with the same gre and gpa but graduated from a prestige =4 college.

> ### Question 16.  Interpret the odds ratio of `gpa`.

In [115]:
1.16279229738-1

0.16279229738

Answer: One incremental grade point will increase the odds of being admitted by approximately 16%.

> ### Question 17.  Assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [116]:
x_prestige_1 = pd.DataFrame({'gre':800, 'gpa': 4, 'prestige_1.0':1, 'prestige_2.0':0, 'prestige_3.0':0}, index=[0])
x_prestige_2 = pd.DataFrame({'gre':800, 'gpa': 4, 'prestige_1.0':0, 'prestige_2.0':1, 'prestige_3.0':0}, index=[0])
x_prestige_3 = pd.DataFrame({'gre':800, 'gpa': 4, 'prestige_1.0':0, 'prestige_2.0':0, 'prestige_3.0':1}, index=[0])
x_prestige_4 = pd.DataFrame({'gre':800, 'gpa': 4, 'prestige_1.0':0, 'prestige_2.0':0, 'prestige_3.0':0}, index=[0])

In [119]:
model.predict(x_prestige_1)

array([  1.67315319e-179])

In [120]:

np.exp(1.67315319e-179)/(1+np.exp(1.67315319e-179))

0.5

## Part D. Moving the model from `statsmodels` to `sklearn`

> ### Question 18.  Let's assume we are satisfied with our model.  Remodel it (same features) using `sklearn`.  When creating the logistic regression model with `LogisticRegression(C = 10 ** 2)`.

In [121]:
model_SK = linear_model.LogisticRegression(C = 10**2).fit(X, y)

> ### Question 19.  What are the odds ratios for the different variables and how do they compare with the odds ratios calculated with `statsmodels`?

In [122]:
print model_SK.intercept_
print model_SK.coef_

[-5.31428638]
[[ 0.00215569  0.76513054  1.50664104  0.84896281  0.18248515]]


In [123]:
odds_ratio(model_SK.coef_)

[[ 1.00215801  2.14927492  4.5115512   2.33722144  1.20019632]]


In [124]:
#Stats model:
odds_ratio(model.params)

gre             1.001069
gpa             0.596794
prestige_1.0    3.650274
prestige_2.0    1.668639
prestige_3.0    0.950821
dtype: float64


Answer: It looks like the ratios in the sklearn model is higher than the ones in the statsmodel.

> ### Question 20.  Again assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [125]:
#Tier1
odds_ratio(model_SK.predict(x_prestige_1))

[ 2.71828183]


In [130]:
model_SK.predict(x_prestige_1)

array([1])

In [126]:
#Tier2
odds_ratio(model_SK.predict(x_prestige_2))

[ 2.71828183]


In [131]:
model_SK.predict(x_prestige_2)

array([1])

In [127]:
#Tier3
odds_ratio(model_SK.predict(x_prestige_3))

[ 2.71828183]


Answer: